<a href="https://colab.research.google.com/github/hamzafarooq/Time-Series/blob/master/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Time series forecasting

In [ ]:
# ray tune dependencies
!pip install -q ray[debug]
!pip install ray[tune]

In [ ]:
# ray tune imports
from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [ ]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
import statsmodels.api as sm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf

## Iowa Dataset
This tutorial uses a <a href="https://console.cloud.google.com/marketplace/details/iowa-department-of-commerce/iowa-liquor-sales" class="external">Iowa Liquor Retails Sales</a>.

This dataset contains every wholesale purchase of liquor in the State of Iowa by retailers for sale to individuals since January 1, 2012. The State of Iowa controls the wholesale distribution of liquor intended for retail sale, which means this dataset offers a complete view of retail liquor sales in the entire state. The dataset contains every wholesale order of liquor by all grocery stores, liquor stores, convenience stores, etc., with details about the store and location, the exact liquor brand and size, and the number of bottles ordered.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
# Save output in a variable `df`

%%bigquery --project bold-sorter-281506 df
SELECT 
  * 
FROM `bigquery-public-data.iowa_liquor_sales.sales`
where store_number  = '2633'
and date > '2018-01-01'

Let's take a glance at the data.

In [ ]:
df.head()

In [ ]:
df.describe()
from datetime import datetime

In [ ]:
df_single_item_aggregate =df[['date','sale_dollars']]
df_single_item_aggregate['date'] = pd.to_datetime(df_single_item_aggregate['date'])
#print(type(date_object))
#print(date_object) 

In [ ]:
df_single_item_aggregate = df_single_item_aggregate.groupby(['date']).sum().rename_axis('date')

In [ ]:
#df_single_item_aggregate['flag'] = pd.Series(np.where(df_single_item_aggregate.index >= np.datetime64('2020-01-25'), 1, 0),index=df_single_item_aggregate.index)
df_single_item_aggregate


In [ ]:
def split_data(data, split_date):
    return data[data.index <= split_date].copy(), \
           data[data.index >  split_date].copy()

In [ ]:
train, test = split_data(df_single_item_aggregate, '2020-06-15')

plt.figure(figsize=(20,10))
plt.xlabel('time')
plt.ylabel('close')
plt.plot(train.index,train)
plt.plot(test.index,test)
plt.show()


In [ ]:
train.describe()

# xgboost Model

In [ ]:
def create_features(df):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    df['flag'] = pd.Series(np.where(df['date'] >= np.datetime64('2020-03-03'), 1, 0), index=df.index)
    
    X = df[['dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','flag']]
    return X

In [ ]:
X_train, y_train = create_features(train), train['sale_dollars']
X_test, y_test   = create_features(test), test['sale_dollars']

X_train.shape, y_train.shape

In [ ]:
X_train.head()

#

In [ ]:
#df['flag'] = pd.Series(np.where(df['date'] >= np.datetime64('2020-01-25'), 1, 0), index=df.index)
X_train.tail()

In [ ]:
params = {
    # Parameters that we are going to tune.
    'colsample_bytree': 1.0,
      'eta': 0.01,
    'eval_metric': 'mae',
 'max_depth': 10,
 'min_child_weight': 6,
 'objective': 'reg:linear',
 'subsample': 0.8,
    'objective':'reg:linear',
}

params2 = {'n_estimator':1000,'subsample': 0.39, 'min_child_weight': 0.3511, 'max_depth': 3, 
           'learning_rate': 0.033, 'gamma': 6.925, 'colsample_bytree': 0.7999999999999996, 'colsample_bylevel': 1.0}


para3={'n_estimator':1000,'subsample': 0.59, 'min_child_weight': 0.21309999999999998, 'max_depth': 4, 'learning_rate': 0.049, 'gamma': 16.125, 
       'colsample_bytree': 0.8799999999999996, 'colsample_bylevel': 1.0}

para4={'subsample': 0.29000000000000004, 'min_child_weight': 0.24109999999999998, 'max_depth': 4, 'learning_rate': 0.045, 
       'gamma': 15.870000000000001, 'colsample_bytree': 0.6399999999999997, 'colsample_bylevel': 1.0}
para5= {'n_estimator':1000,'subsample': 0.33, 'min_child_weight': 0.4021, 'max_depth': 3, 'learning_rate': 0.0455, 
        'gamma': 4.43, 'colsample_bytree': 0.9499999999999995, 'colsample_bylevel': 1.0}


para5['eval_metric'] = "mae"
num_boost_round = 999

In [ ]:
#reg = xgb.XGBRegressor(n_estimators=1000,subsample=0.59,min_child_weight=0.213099,max_depth=4,learning_rate=0.049,gamma=16.125,colsample_bytree=0.87999,
 #                      colsample_bylevel=1.0)
reg=xgb.XGBRegressor(**para5)

reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=500, #stop if 50 consequent rounds without decrease of error
        verbose=False
        )
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
model = xgb.train(
    para5,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=100,verbose_eval=False,
)
 # Change verbose to True if you want to see it train

In [ ]:
X_test_pred = reg.predict(X_test)
X_test_pred_2=model.predict(dtest)

In [ ]:
def plot_performance(base_data, date_from, date_to, title=None):
    plt.figure(figsize=(20,5))
    if title == None:
        plt.title('From {0} To {1}'.format(date_from, date_to))
    else:
        plt.title(title)
    plt.xlabel('time')
    plt.ylabel('close')
    plt.plot(df_single_item_aggregate.index,df_single_item_aggregate, label='data')
    plt.plot(X_test.index,X_test_pred, label='prediction')
    plt.plot(X_test.index,X_test_pred_2, label='prediction_2')
    plt.legend()
    plt.xlim(left=date_from, right=date_to)

In [ ]:
#xgb.plot_importance(reg, height=0.9)
#X_test_pred = reg.predict(X_test)
    
plot_performance(df_single_item_aggregate, df_single_item_aggregate.index[0].date(), df_single_item_aggregate.index[-1].date(),
                 'Original and Predicted Data')

plot_performance(y_test, y_test.index[0].date(), y_test.index[-1].date(),
                 'Test and Predicted Data')

#plot_performance(y_test, '2019-7-01', '2019-8-01', 'Snapshot')

plt.legend()

plt.show()

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mean_absolute_percentage_error(y_test,X_test_pred)

In [ ]:

def calc_smape(y_hat, y):
        return 100/len(y) * np.sum(2 * np.abs(y_hat - y) / (np.abs(y) + np.abs(y_hat)))

In [ ]:
calc_smape(y_test,X_test_pred)

In [306]:
calc_smape(y_test,X_test_pred_2)

72.09748633386053

In [ ]:
error_by_week = []
random_weeks = X_test[['year', 'weekofyear']].sample(10)
for week in random_weeks.iterrows():
    index = (X_test.year == week[1].year) & \
            (X_test.weekofyear == week[1].weekofyear)
    error_by_week.append(mean_absolute_percentage_error(y_test[index], X_test_pred[index]))
pd.Series(error_by_week, index=random_weeks.index)

In [ ]:
sklearn.metrics.SCORERS.keys()

#Hyperparameter Tuning

In [ ]:
%%capture
import time
import numpy as np
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
#from sklearn.metrics import accuracy
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

start_time=time.time()

#### Create X and Y training data here.....


# grid search
model = XGBRegressor()

param_grid = {
        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'min_child_weight': np.arange(0.0001, 0.5, 0.001),
        'gamma': np.arange(0.0,40.0,0.005),
        'learning_rate': np.arange(0.0005,0.3,0.0005),
        'subsample': np.arange(0.01,1.0,0.01),
        'colsample_bylevel': np.round(np.arange(0.1,1.0,0.01)),
        'colsample_bytree' : np.arange(0.1,1.0,0.01)
}

kfold = KFold(n_splits=10, shuffle=True, random_state=10)
grid_search = RandomizedSearchCV(model, param_grid, scoring="r2", n_iter = 1000, cv=kfold)
grid_result = grid_search.fit(X_train,y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_[ 'mean_test_score' ]
stds = grid_result.cv_results_[ 'std_test_score' ]
params = grid_result.cv_results_[ 'params' ]

print(time.time()-start_time)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_[ 'mean_test_score' ]
stds = grid_result.cv_results_[ 'std_test_score' ]
params = grid_result.cv_results_[ 'params' ]

print(time.time()-start_time)

In [ ]:
import numpy as np
import sklearn.datasets
import sklearn.metrics
from ray.tune.schedulers import ASHAScheduler
from sklearn.model_selection import train_test_split

In [ ]:
def XGBCallback(env):
    # After every training iteration, report loss to Tune
    tune.report(**dict(env.evaluation_result_list))

In [ ]:
# X_train_numpy = X_train.to_numpy()
# X_test_numpy = X_test.to_numpy()
# y_train_numpy = np.reshape(y_train.to_numpy(), (y_train.shape[0], 1))
# y_test_numpy = np.reshape(y_test.to_numpy(), (y_test.shape[0], 1))

In [ ]:
#X_train_numpy[1]

In [ ]:
X_train.head()

In [ ]:
def plot_performance(base_data, date_from, date_to, title=None):
    plt.figure(figsize=(15,3))
    if title == None:
        plt.title('From {0} To {1}'.format(date_from, date_to))
    else:
        plt.title(title)
    plt.xlabel('time')
    plt.ylabel('close')
    plt.plot(df_single_item_aggregate.index,df_single_item_aggregate, label='data')
    plt.plot(X_test.index,preds, label='prediction')
    plt.legend()
    plt.xlim(left=date_from, right=date_to)

In [ ]:
l=[]
def train_forecaster_tune(config):
    X_train_numpy = X_train.to_numpy()
    X_test_numpy = X_test.to_numpy()
    y_train_numpy = np.reshape(y_train.to_numpy(), (y_train.shape[0], 1))
    y_test_numpy = np.reshape(y_test.to_numpy(), (y_test.shape[0], 1))

    train_set = xgb.DMatrix(X_train_numpy, label=y_train_numpy)
    test_set = xgb.DMatrix(X_test_numpy, label=y_test_numpy)

    # Train the classifier
    bst = xgb.train(
        config,
        train_set,
        evals=[(test_set, "eval")],
        verbose_eval=False,
        callbacks=[XGBCallback])
    # Predict labels for the test set
    preds = bst.predict(test_set)
    l.append(preds)

    # pred_labels = np.rint(preds)
    # Return prediction accuracy
    # accuracy = sklearn.metrics.accuracy_score(test_y, pred_labels)
    # tune.report(mean_accuracy=accuracy, done=True)
    score = calc_smape(y_test_numpy, preds)
    print(preds)
    print(f'the accuracy is {score}')
    tune.report(mean_accuracy=score, done=True)
    plot_performance(y_test, y_test.index[0].date(), y_test.index[-1].date(),
                 'Test and Predicted Data')

In [ ]:
config = {
  
  "objective": "reg:squarederror",
  "max_depth": tune.randint(1, 10),
  "min_child_weight": tune.choice([1, 2, 3,4,5,6]),
  "subsample": tune.uniform(0.5, 1.0),
  "eta": tune.loguniform(1e-4, 1e-1),
  "eval_metric": [ "rmse"],
  "n_estimators": 10000

}
param_grid = {
        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'min_child_weight': np.arange(0.0001, 0.5, 0.001),
        'gamma': np.arange(0.0,40.0,0.005),
        'learning_rate': np.arange(0.0005,0.3,0.0005),
        'subsample': np.arange(0.01,1.0,0.01),
        'colsample_bylevel': np.round(np.arange(0.1,1.0,0.01)),
        'colsample_bytree' : np.arange(0.1,1.0,0.01)
}



# The ASHAScheduler stops bad performing configurations early
scheduler = ASHAScheduler(
  metric="eval-rmse",  # The `eval` prefix is defined in xgb.train
  mode="max",  # Retain configurations with a low logloss
  max_t=11,  # 10 training iterations + 1 final evaluation
  grace_period=1,  # Number of minimum iterations for each trial
  reduction_factor=2)  # How aggressively to stop trials
analysis = tune.run(
  train_forecaster_tune,  # your training function
  resources_per_trial={"gpu": 0.1},  # You can add "gpu": 0.1 here
  config=config,
  num_samples=10,  # number of parameter configurations to try
  scheduler=scheduler)

In [ ]:

best_config = analysis.get_best_logdir('mean_accuracy')

In [ ]:
str(best_config)

In [ ]:
best_config

In [ ]:
analysis.runner_data()

In [ ]:
l